This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
#import os
#from dotenv import load_dotenv

#load_dotenv()
#API_KEY = os.getenv('')

#print(API_KEY)

API_KEY = ''

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests 

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2014-01-01&end_date=2014-01-02&api_key="+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2014-01-01',
  'end_date': '2014-01-02',
  'frequency': 'daily',
  'data': [['2014-01-02',
    24.24,
    24.76,
    23.77,
    24.18,
    None,
    55735.0,
    1355658.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
response = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2018-01-01&api_key="+API_KEY)

In [6]:
# 2. Convert the returned JSON object into a Python dictionary.
json = response.json() 

data = json['dataset_data']['data'][::-1] # Reverses list from reverse-chronological to chronological order

col_names = json['dataset_data']['column_names'] # Collect the column names
print(col_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [7]:
data_dict = {k:[] for k in col_names} # Convert the JSON to a Python dictionary

for i, col in enumerate(col_names):
    for day in data:
        data_dict[col].append(day[i])
        
print(type(data_dict))

<class 'dict'>


In [8]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
highest_price = 0.0 
lowest_price = 1000.0

for _open, _close in zip(data_dict['Open'], data_dict['Close']):
    if _open: #ignores three `None` values
        if _open > highest_price:
            highest_price = _open
    
    if _close: #ignores three `None` values
        if _close < lowest_price:
            lowest_price = _close
            
print('The highest opening price was $' + str(highest_price) + '.')
print('The lowest opening price was $' + str(lowest_price) + '.')

The highest opening price was $53.11.
The lowest opening price was $34.06.


In [9]:
# 4. What was the largest change in any one day (based on High and Low price)?
high_low_deltas = []

for hi, lo in zip(data_dict['High'], data_dict['Low']): 
    delta = hi - lo
    high_low_deltas.append(round(delta, 2))

print('The largest change in any one day was $' + str(max(high_low_deltas)) + '.')

The largest change in any one day was $2.81.


In [10]:
# 5. What was the largest change between any two days (based on Closing Price)?
close_prices = []

for day in data_dict['Close']:
    close_prices.append(day)

In [11]:
close_deltas = []

for i in range(len(close_prices)):
    if i != 0: #ignores first date as there is no previous closing price to compare
        delta = close_prices[i] - close_prices[i-1]
        close_deltas.append(round(delta, 2))

print('The largest change between any two adjacent closing prices was $' + str(max(close_deltas)) + '.')

The largest change between any two adjacent closing prices was $1.72.


In [12]:
# 6. What was the average daily trading volume during this year?
avg_traded_vol = sum(data_dict['Traded Volume']) / len(data_dict['Traded Volume'])
print('The average daily traded volume was ' + str(int(avg_traded_vol)) + ' units.') 

The average daily traded volume was 89124 units.


In [13]:
# 7. (Optional) What was the median trading volume during this year?
# (Note: you may need to implement your own function for calculating the median.)

sorted_traded_vol = sorted(data_dict['Traded Volume'])
# len(sorted_traded_vol) ---> 255 / 2 = 127.5

med_traded_vol = (sorted_traded_vol[128] + sorted_traded_vol[127]) / 2
print('The median daily traded volume was ' + str(int(med_traded_vol)) + ' units.')

The median daily traded volume was 76443 units.
